# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

### Import Dependencies

In [1]:
from azureml.core.workspace import Workspace
import logging
from azureml.train.automl import AutoMLConfig
from azureml.core.experiment import Experiment
from azureml.widgets import RunDetails 


### Workspace configuration

In [2]:
ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'capstone_automl'

experiment=Experiment(ws, experiment_name)

### Compute Configuration

In [3]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

compute_cluster_name = 'capstone-compute'

try:
    compute_target = ComputeTarget(workspace=ws, name = compute_cluster_name)
    print('Found the compute cluster')

except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size="STANDARD_D3_V2", max_nodes=6)
    compute_target = ComputeTarget.create(ws, compute_cluster_name, compute_config)
    compute_target.wait_for_completion(show_output=True)

Creating...
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.

#### Dataset Overview
Data is taken from kaggle Datasets collection for price fix of newly manufactured mobiles in company.

#### Task:
Estimate the price of mobiles produced by company based on sales data of mobile phones of various companies.
That is find out some relation between features of a mobile phone(eg:- RAM,Internal Memory etc) and decide price based on it.

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [4]:
from azureml.core.dataset import Dataset
from sklearn.model_selection import train_test_split

key = 'mobile price classification dataset'
if key in ws.datasets.keys():
    dataset = ws.datasets[key]
    print("dataset found!")

else:
    mobile_sales_data = Dataset.get_by_name(ws, name='mobile_data')

In [5]:
mobile_sales_data.take(3).to_pandas_dataframe()
#mobile_sales_data

,battery_power,blue,clock_speed,dual_sim,fc,four_g,int_memory,m_dep,mobile_wt,n_cores,...,px_height,px_width,ram,sc_h,sc_w,talk_time,three_g,touch_screen,wifi,price_range
0,842,0,2.2,0,1,0,7,0.6,188,2,...,20,756,2549,9,7,19,0,0,1,1
1,1021,1,0.5,1,0,1,53,0.7,136,3,...,905,1988,2631,17,3,7,1,1,0,2
2,563,1,0.5,1,2,1,41,0.9,145,5,...,1263,1716,2603,11,2,9,1,1,0,2


## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [6]:
# TODO: Put your automl settings here
automl_settings = {"iteration_timeout_minutes": 7,
    "experiment_timeout_minutes": 25,
    "enable_early_stopping": True,
    "primary_metric": 'accuracy',
    "featurization": 'auto',
    "verbosity": logging.INFO,
    "n_cross_validations": 4}

# TODO: Put your automl config here
automl_config = AutoMLConfig(compute_target = compute_target,
                             task='classification',
                             debug_log='automated_ml_errors.log',
                             training_data=mobile_sales_data,
                             label_column_name="price_range",
                             **automl_settings)

In [7]:
# TODO: Submit your experiment
remote_run = experiment.submit(automl_config)


Submitting remote run.


Experiment,Id,Type,Status,Details Page,Docs Page
capstone_automl,AutoML_e5dc7447-e4a9-44ac-a048-0ebec61dd69e,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation


## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [8]:
RunDetails(remote_run).show()
remote_run.wait_for_completion(show_output=True)

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

Experiment,Id,Type,Status,Details Page,Docs Page
capstone_automl,AutoML_e5dc7447-e4a9-44ac-a048-0ebec61dd69e,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation



Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

****************************************************************************************************

TYPE:         Missing feature values imputation
STATUS:       PASSED
DESCRIPTION:  No feature missing values were detected in the training data.
              Learn more about missing value imputation: https://aka.ms/AutomatedMLFeaturization

******************************************************************

{'runId': 'AutoML_e5dc7447-e4a9-44ac-a048-0ebec61dd69e',
 'target': 'capstone-compute',
 'status': 'Completed',
 'startTimeUtc': '2021-04-20T01:37:30.904856Z',
 'endTimeUtc': '2021-04-20T02:13:09.421211Z',
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'accuracy',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '4',
  'target': 'capstone-compute',
  'AMLSettingsJsonString': '{"path":null,"name":"capstone_automl","subscription_id":"a0a76bad-11a1-4a2d-9887-97a29122c8ed","resource_group":"aml-quickstarts-143196","workspace_name":"quick-starts-ws-143196","region":"southcentralus","compute_target":"capstone-compute","spark_service":null,"azure_service":"remote","many_models":false,"pipeline_fetch_max_batch_size":1,"enable_batch_run":false,"enable_run_restructure":false,"iterations":1000,"primary_metric":"accuracy","task_type":"classification","data_script":null,"test_size":0.0,"val

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [9]:
best_run, model = remote_run.get_output()
print(best_run)
print(model)

Run(Experiment: capstone_automl,
Id: AutoML_e5dc7447-e4a9-44ac-a048-0ebec61dd69e_19,
Type: azureml.scriptrun,
Status: Completed)
Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=None, enable_feature_sweeping=None,
                                 feature_sweeping_config=None,
                                 feature_sweeping_timeout=None,
                                 featurization_config=None, force_text_dnn=None,
                                 is_cross_validation=None,
                                 is_onnx_compatible=None, logger=None,
                                 observer=None, task=None, working_dir=None)),
                ('prefittedsoftvotingclassifier',...
                                                                                                    n_estimators=10,
                                                                                                    n_jobs=1,
                                    

In [10]:
best_run.get_metrics(name='accuracy')

{'accuracy': 0.9375}

In [11]:
best_run_metrics = best_run.get_metrics()
best_run_metrics

{'accuracy': 0.9375,
 'balanced_accuracy': 0.9372973651295377,
 'matthews_correlation': 0.9168088244657683,
 'AUC_micro': 0.9943833333333334,
 'norm_macro_recall': 0.9163964868393835,
 'precision_score_macro': 0.9375269192742518,
 'precision_score_weighted': 0.9376282585231983,
 'AUC_macro': 0.9936597163698846,
 'f1_score_weighted': 0.9372410485200529,
 'precision_score_micro': 0.9375,
 'recall_score_weighted': 0.9375,
 'log_loss': 0.5014385077550394,
 'weighted_accuracy': 0.9377126255665977,
 'f1_score_macro': 0.9370929842806404,
 'AUC_weighted': 0.9936656276629385,
 'average_precision_score_weighted': 0.981582048420412,
 'average_precision_score_micro': 0.9836532799755314,
 'f1_score_micro': 0.9375,
 'recall_score_macro': 0.9372973651295377,
 'average_precision_score_macro': 0.9815599942815594,
 'recall_score_micro': 0.9375,
 'accuracy_table': 'aml://artifactId/ExperimentRun/dcid.AutoML_e5dc7447-e4a9-44ac-a048-0ebec61dd69e_19/accuracy_table',
 'confusion_matrix': 'aml://artifactId/Ex

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [23]:
#TODO: Save the best model
from azureml.core import Model

# Register model
auto_model=best_run.register_model(model_path='outputs/model.pkl', model_name='automl_model',
                        tags={'Training context':'Auto ML'},
                        properties={'AUC': best_run_metrics['AUC_weighted'], 'Accuracy': best_run_metrics['accuracy']})

# List registered models
for model in Model.list(ws):
    print(model.name, 'version:', model.version)
    for tag_name in model.tags:
        tag = model.tags[tag_name]
        print ('\t',tag_name, ':', tag)
    for prop_name in model.properties:
        prop = model.properties[prop_name]
        print ('\t',prop_name, ':', prop)

automl_model version: 2
	 Training context : Auto ML
	 AUC : 0.9936656276629385
	 Accuracy : 0.9375
automl_model version: 1
	 Training context : Auto ML
	 AUC : 0.9936656276629385
	 Accuracy : 0.9375
best_hd_run version: 3
best_hd_run version: 2
best_hd_run version: 1


In [25]:
from azureml.automl.core.shared import constants
best_run.download_file(constants.CONDA_ENV_FILE_PATH, 'myenv.yml')

In [26]:
# create inference_config
from azureml.core.model import InferenceConfig
from azureml.core.environment import Environment
myenv = Environment.from_conda_specification(name = 'myenv',
                                             file_path = 'myenv.yml')
inference_config = InferenceConfig(entry_script="scoring.py",environment=myenv)

In [27]:
from azureml.core.webservice import Webservice, AciWebservice
deployment_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb = 1)

TODO: In the cell below, send a request to the web service you deployed to test it.

In [28]:
#deploy the model
service=Model.deploy(workspace=ws,
                    name="mobile-price-range-classifier",
                    models=[auto_model],
                    inference_config=inference_config,
                    deployment_config=deployment_config)
service.wait_for_deployment(show_output=True)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2021-04-20 02:58:29+00:00 Creating Container Registry if not exists.
2021-04-20 02:58:29+00:00 Registering the environment.
2021-04-20 02:58:30+00:00 Use the existing image.
2021-04-20 02:58:31+00:00 Generating deployment configuration.
2021-04-20 02:58:33+00:00 Submitting deployment to compute..
2021-04-20 02:58:38+00:00 Checking the status of deployment mobile-price-range-classifier..
2021-04-20 03:03:53+00:00 Checking the status of inference endpoint mobile-price-range-classifier.
Succeeded
ACI service creation operation finished, operation "Succeeded"


In [29]:
print(service.get_logs())

2021-04-20T03:03:42,351607400+00:00 - iot-server/run 
2021-04-20T03:03:42,353835400+00:00 - rsyslog/run 
2021-04-20T03:03:42,363460800+00:00 - gunicorn/run 
2021-04-20T03:03:42,393115900+00:00 - nginx/run 
rsyslogd: /azureml-envs/azureml_730341d347b7d9e2f1baba033329cdff/lib/libuuid.so.1: no version information available (required by rsyslogd)
/usr/sbin/nginx: /azureml-envs/azureml_730341d347b7d9e2f1baba033329cdff/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_730341d347b7d9e2f1baba033329cdff/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_730341d347b7d9e2f1baba033329cdff/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_730341d347b7d9e2f1baba033329cdff/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml

In [30]:
scoring_uri = service.scoring_uri

print(f'\nservice state: {service.state}\n')
print(f'scoring URI: \n{service.scoring_uri}\n')
print(f'swagger URI: \n{service.swagger_uri}\n')

print(service.scoring_uri)
print(service.swagger_uri)


service state: Healthy

scoring URI: 
http://2c1376e2-19cf-42e1-9eb8-e3c4a2bc53c8.southcentralus.azurecontainer.io/score

swagger URI: 
http://2c1376e2-19cf-42e1-9eb8-e3c4a2bc53c8.southcentralus.azurecontainer.io/swagger.json

http://2c1376e2-19cf-42e1-9eb8-e3c4a2bc53c8.southcentralus.azurecontainer.io/score
http://2c1376e2-19cf-42e1-9eb8-e3c4a2bc53c8.southcentralus.azurecontainer.io/swagger.json


TODO: In the cell below, print the logs of the web service and delete the service

In [31]:
x = mobile_sales_data.to_pandas_dataframe().iloc[0, :-1].to_dict()
y = mobile_sales_data.to_pandas_dataframe().iloc[0, -1]

In [32]:
print(x)

{'battery_power': 842.0, 'blue': 0.0, 'clock_speed': 2.2, 'dual_sim': 0.0, 'fc': 1.0, 'four_g': 0.0, 'int_memory': 7.0, 'm_dep': 0.6, 'mobile_wt': 188.0, 'n_cores': 2.0, 'pc': 2.0, 'px_height': 20.0, 'px_width': 756.0, 'ram': 2549.0, 'sc_h': 9.0, 'sc_w': 7.0, 'talk_time': 19.0, 'three_g': 0.0, 'touch_screen': 0.0, 'wifi': 1.0}


In [33]:
print(y)

1


In [34]:
import requests
import json

data = {
  "data": [
    {
                  'battery_power':842,
                  'blue':0,
                  'clock_speed':2.2,
                  'dual_sim':0,
                  'fc':1,
                  'four_g':0,
                  'int_memory':7,
                  'm_dep':0.6,
                  'mobile_wt':188,
                  'n_cores':2,
                  'pc':2,
                  'px_height':20,
                  'px_width':756,
                  'ram':2549,
                  'sc_h':9,
                  'sc_w':7,
                  'talk_time':19,
                  'three_g':0,
                  'touch_screen':0,
                  'wifi':	1
                 
                 
    }
 ]}
    
# Convert to JSON string
input_data = json.dumps(data)
output = service.run(input_data)
headers = {'Content-Type': 'application/json'}

In [35]:
# Make the request and display the response
resp = requests.post(scoring_uri, input_data, headers=headers)
print("prediction ", resp.text)

prediction  [1]


In [36]:
print('Status code:',resp.status_code)
print(service.get_logs())

Status code: 200
2021-04-20T03:03:42,351607400+00:00 - iot-server/run 
2021-04-20T03:03:42,353835400+00:00 - rsyslog/run 
2021-04-20T03:03:42,363460800+00:00 - gunicorn/run 
2021-04-20T03:03:42,393115900+00:00 - nginx/run 
rsyslogd: /azureml-envs/azureml_730341d347b7d9e2f1baba033329cdff/lib/libuuid.so.1: no version information available (required by rsyslogd)
/usr/sbin/nginx: /azureml-envs/azureml_730341d347b7d9e2f1baba033329cdff/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_730341d347b7d9e2f1baba033329cdff/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_730341d347b7d9e2f1baba033329cdff/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_730341d347b7d9e2f1baba033329cdff/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbi

In [ ]:
service.delete()
compute_target.delete()
model.delete()
experiment.delete()